### 2) Exploration des donnees KPI

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Connexion à la base de données
conn = sqlite3.connect('../Datas/db/fraude_detection_warehouse_.db')

### 1. Distribution des Types de Transactions

In [2]:
transactions = pd.read_sql_query('SELECT * FROM transactions', conn)
transactions.head()

,transaction_id,customer_id,device_id,transaction_date,transaction_type,amount,status,is_fraud,location
0,731b17cfadce442ab756b5b110a406c2,d159ae25b3d04387bbc006b8f0df4629,f66d0310-bf70-4552-a732-5950c93b1134,2020-02-12,depôt,9.385805e+06,réussie,1,Douala
1,96f4ac611eaa4174b39149d98e569810,84d0894d1b9a4a8dbe13cfbb32d451df,f765ead3-ca24-48f4-9252-a6fd5fc53518,2022-02-03,depôt,3.358191e+06,réussie,0,Bafoussam
2,1c11b97ba2994efbb6f85291e7062e00,b01c160bdd134ac989f1d0d0163ac149,9cac18d3-defc-4142-9a39-b66fe1063f6e,2019-11-09,depôt,7.307758e+06,réussie,1,Bafoussam
3,99d146a3fbd04a38a5b83190b4ec22e3,31bd9c0aa340439899f52de6d89ac6eb,40ccde62-b98d-4e6b-bd3f-bf514cae73b9,2018-07-18,transfert,3.127519e+06,réussie,1,Bafoussam
4,bdab6a6430b44fadbf2f1682ccda4ae8,ade32958209d40cfb475e5bef5a2b4a3,16aa8a04-4cbd-48dc-88d2-ef9d0e8a2944,2019-12-29,retrait,4.435274e+06,réussie,0,Yaoundé


In [3]:
# Load transaction data
transactions = pd.read_sql_query('SELECT * FROM transactions', conn)

# Nombre total de transactions par type
transaction_types = pd.read_sql_query("""
    SELECT transaction_type, COUNT(*) AS total_transactions
    FROM transactions
    GROUP BY transaction_type
""", conn)

# Pourcentage de chaque type de transaction par rapport au total des transactions.
transaction_types['percentage'] = (transaction_types['total_transactions'] / transaction_types['total_transactions'].sum()) * 100

print(transaction_types)

  transaction_type  total_transactions  percentage
0            depôt               50048      50.048
1          retrait               24896      24.896
2        transfert               25056      25.056


### 2. Statistiques des Montants des Transactions

### 3. Analyse Temporelle

In [4]:
# Nombre de transactions par jour
daily_transactions = pd.read_sql_query('SELECT DATE(transaction_date) as date, COUNT(*) as transaction_count FROM transactions GROUP BY date', conn)
#print(daily_transactions)

# Montant total des transactions par jour
daily_amounts = pd.read_sql_query('SELECT DATE(transaction_date) as date, SUM(amount) as total_amount FROM transactions GROUP BY date', conn)
#print(daily_amounts)

# Heure de la journée la plus fréquente pour les transactions
transactions['hour'] = pd.to_datetime(transactions['transaction_date']).dt.hour
most_frequent_hour = transactions['hour'].mode()[0]
print(f"Heure de la journée la plus fréquente pour les transactions: {most_frequent_hour}")

Heure de la journée la plus fréquente pour les transactions: 0


### 4. Analyse par Localisation

In [5]:
# Nombre de transactions par région
regional_transactions = pd.read_sql_query('SELECT location, COUNT(*) as transaction_count FROM transactions GROUP BY location', conn)
#print(regional_transactions)

# Montant total des transactions par région
regional_amounts = pd.read_sql_query('SELECT location, SUM(amount) as total_amount FROM transactions GROUP BY location', conn)
#print(regional_amounts)

# Distribution des types de transactions par région
regional_distribution = pd.read_sql_query('SELECT location, transaction_type, COUNT(*) as count FROM transactions GROUP BY location, transaction_type', conn)
#print(regional_distribution)

### 5. Analyse des Clients

In [6]:
# Load customers data
customers = pd.read_sql_query('SELECT * FROM customers', conn)

# Nombre total de clients actifs
active_customers = pd.read_sql_query('SELECT COUNT(DISTINCT customer_id) as active_customers FROM transactions', conn)
#print(active_customers)

# Distribution des âges des clients
customers['age'] = pd.to_datetime(customers['date_of_birth']).dt.year
age_distribution = customers['age'].value_counts().sort_index()
#print(age_distribution)

# Nombre moyen de transactions par client
avg_transactions_per_customer = pd.read_sql_query('SELECT AVG(transaction_count) as avg_transactions FROM (SELECT customer_id, COUNT(*) as transaction_count FROM transactions GROUP BY customer_id)', conn)
#print(avg_transactions_per_customer)

# Montant moyen des transactions par client
avg_amount_per_customer = pd.read_sql_query('SELECT AVG(amount) as avg_amount FROM transactions GROUP BY customer_id', conn)
#print(avg_amount_per_customer)

### 6. Analyse des Appareils

In [7]:
# Distribution des types d'appareils utilisés
device_distribution = pd.read_sql_query('SELECT device_type, COUNT(*) as count FROM devices GROUP BY device_type', conn)
#print(device_distribution)

# Distribution des systèmes d'exploitation
os_distribution = pd.read_sql_query('SELECT os, COUNT(*) as count FROM devices GROUP BY os', conn)
#print(os_distribution)

# Nombre de transactions par appareil
transactions_per_device = pd.read_sql_query('SELECT d.device_type, COUNT(t.transaction_id) as transaction_count FROM transactions t JOIN devices d ON t.device_id = d.device_id GROUP BY d.device_type', conn)
#print(transactions_per_device)

# Montant total des transactions par appareil
total_amount_per_device = pd.read_sql_query('SELECT d.device_type, SUM(t.amount) as total_amount FROM transactions t JOIN devices d ON t.device_id = d.device_id GROUP BY d.device_type', conn)
#print(total_amount_per_device)

### 7. Analyse des Historique de Transactions

In [8]:
# Requête SQL pour compter le nombre total de transactions
total_transactions = pd.read_sql_query("""
SELECT COUNT(*) as total_transactions
FROM transactions
""", conn)

# Affichage du résultat
print("Nombre total de transactions :")
print(total_transactions['total_transactions'][0])

Nombre total de transactions :
100000


### 8. Détection de Fraude

In [9]:
# Nombre total de transactions frauduleuses
total_fraudulent_transactions = pd.read_sql_query('SELECT COUNT(*) as total_fraud FROM transactions WHERE is_fraud = 1', conn)
#print(total_fraudulent_transactions)

# Pourcentage de transactions frauduleuses par rapport au total des transactions
#fraud_percentage = (total_fraudulent_transactions['total_fraud'][0] / total_transactions) * 100

# Montant total des transactions frauduleuses
total_fraud_amount = pd.read_sql_query('SELECT SUM(amount) as total_fraud_amount FROM transactions WHERE is_fraud = 1', conn)
#print(total_fraud_amount)

# Distribution des transactions frauduleuses par type, région, appareil, et période
fraud_distribution = pd.read_sql_query('SELECT transaction_type, location, device_id, COUNT(*) as count FROM transactions WHERE is_fraud = 1 GROUP BY transaction_type, location, device_id', conn)
#print(fraud_distribution)

# Taux de fraude par client
fraud_rate_per_customer = pd.read_sql_query('SELECT customer_id, COUNT(*) as fraud_count FROM transactions WHERE is_fraud = 1 GROUP BY customer_id', conn)
#print(fraud_rate_per_customer)


In [10]:
total_fraudulent_transactions

,total_fraud
0,56707


### 9. Alertes de Fraude

In [ ]:
# Nombre total d'alertes générées
total_alerts = pd.read_sql_query('SELECT COUNT(*) as total_alerts FROM alerts', conn)
#print(total_alerts)

# Distribution des types d'alertes
alert_distribution = pd.read_sql_query('SELECT alert_type, COUNT(*) as count FROM alerts GROUP BY alert_type', conn)
#print(alert_distribution)

# Taux de résolution des alertes
resolution_rate = pd.read_sql_query('SELECT (SUM(CASE WHEN alert_status = "closed" THEN 1 ELSE 0 END) * 1.0 / COUNT(*)) as resolution_rate FROM alerts', conn)
#print(resolution_rate)

# Temps moyen de résolution des alertes
average_resolution_time = pd.read_sql_query('SELECT AVG(julianday(alert_date) - julianday(transaction_date)) as avg_resolution_time FROM alerts JOIN transactions ON alerts.transaction_id = transactions.transaction_id', conn)
#print(average_resolution_time)

### 10. Cas de Fraude Confirmés